In [1]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from dateutil.relativedelta import relativedelta
from datetime import datetime
from dateutil.parser import parse
import copy

from imblearn.over_sampling import SMOTE ,RandomOverSampler
#시각화
import matplotlib.pyplot as plt
import seaborn as sns


from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import *
from sklearn.metrics import accuracy_score, confusion_matrix, log_loss

##알고리즘 
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

In [2]:
# 다양한 오차 측정 지표를 확인하기 위한 함수 정의

from sklearn.metrics import *

def get_clf_eval(y_test, y_data, pred=None, pred_proba=None): 
    confusion = confusion_matrix(y_test, pred) 
    accuracy = accuracy_score(y_test, pred) 
    precision = precision_val(y_data)
    recall = recall_val(y_data) 
    f1 = f1_score(y_test, pred , average= 'weighted') 
    return [accuracy, precision, recall, f1]

In [3]:
def Cut(Series , cuts):
    "구분할 iter가능한 변수와 구분 기준을 입력받으면 구분 기준 앞에서 부터 1로 구분해서 return 함"
    Cuts = copy.deepcopy(cuts)
    Cuts.append(np.inf)
    Cuts.insert(0,-1*np.inf)
    R = len(Cuts)
    label = range(1,R)
    return pd.cut(Series, Cuts, labels = label)

In [4]:
def precision_val(y_data):
    

    new = y_data[y_data['예측 y 라벨링'] > 2]
    real = new[new['실제 y라벨링'] > 2]
    if len(new) == 0:
        per = 0
    else :
        per = len(real)/len(new)
    return per

In [5]:
def recall_val(y_data):

    real = y_data[y_data['실제 y라벨링'] > 2]
    new = real[real['예측 y 라벨링'] > 2]
    if len(new) == 0:
        per = 0
    else :
        per = len(new)/len(real)
    return per

In [6]:
df_1_3 = pd.read_csv("d:/공모전/IPO/IPO/CleanData/X_train_After_IPO_1.csv",encoding = "cp949")
df_3_6 = pd.read_csv("d:/공모전/IPO/IPO/CleanData/X_train_After_IPO_3.csv",encoding = "cp949")
df_0 = pd.read_csv("d:/공모전/IPO/IPO/CleanData/X_train_Before_IPO.csv",encoding = "cp949")
df_end = pd.read_csv("d:/공모전/IPO/IPO/CleanData/X_train_EndPrice_IPO.csv",encoding = "cp949")

In [7]:
df_0_1 = df_0.drop(['공모가 대비 3개월 수익률','공모가 대비 6개월 수익률'],axis =1)
df_0_3 = df_0.drop(['공모가 대비 1개월 수익률','공모가 대비 6개월 수익률'],axis =1)
df_0_6 = df_0.drop(['공모가 대비 1개월 수익률','공모가 대비 3개월 수익률'],axis =1)

In [8]:
df_end_1 = df_end.drop(['종가 대비 3개월 수익률','종가 대비 6개월 수익률'],axis =1)
df_end_3 = df_end.drop(['종가 대비 1개월 수익률','종가 대비 6개월 수익률'],axis =1)
df_end_6 = df_end.drop(['종가 대비 1개월 수익률','종가 대비 3개월 수익률'],axis =1)

# 랜덤포레스트 값

In [182]:
from sklearn.preprocessing import StandardScaler

In [231]:
def process_final(df,y_name):
    
    ## 기본 전처리
    df['상장일'] = pd.to_datetime(df['상장일'])
    df = df.set_index(['상장일'])
    df = df.drop(['종목명','공모 시가총액'],axis = 1) ## 나중에 카테고리 진행할려면 남겨줄 것
    if y_name == '공모가 대비 6개월 수익률' or "종가 대비 6개월 수익률":
        df['Cat'] = Cut(df[y_name],[ -0.2, 0.2, 0.4])
       
        
    else :
        df['Cat'] = Cut(df[y_name],[ -0.1, 0.1, 0.2])
       
    
    ## train 3년 test 1년으로 총 기간을 3개월 이동으로 36개 구간분할
    train_list = []
    test_list = []
    train_start_date = '2009-04-01' ## 기한은 나중에 변경할수도
    test_start_date = parse(str(train_start_date)).date() + relativedelta(years =4)
    train_end_date = parse(str(test_start_date)).date() - relativedelta(days =1)
    test_end_date = parse(str(train_end_date)).date() + relativedelta(years =1) 

    while True:

        train_list.append(df[train_start_date : train_end_date])
        test_list.append(df[test_start_date : test_end_date])

        train_start_date = parse(str(train_start_date)).date() + relativedelta(months=3)
        test_start_date = parse(str(test_start_date)).date() + relativedelta(months=3)
        train_end_date = parse(str(train_end_date)).date() + relativedelta(months=3)
        test_end_date = parse(str(test_end_date)).date() + relativedelta(months=3)

        if str(train_start_date) == '2017-04-01':
            break
            
    y_data = []
    score_list = []
    feature_list = []

    for i in range(0,len(train_list)):
        X_train_before = train_list[i].drop([y_name,'Cat'],axis =1)
        y_train_before = train_list[i]['Cat']
        X_test = test_list[i].drop([y_name,'Cat'],axis =1)
        y_test = test_list[i]['Cat']
        

        
        smote = SMOTE(random_state=0,k_neighbors = 3)
        X_train,y_train = smote.fit_resample(X_train_before,y_train_before)
#         ros = RandomOverSampler()
#         X_train,y_train = ros.fit_resample(X_train_before,y_train_before)

       # 랜덤 포레스트 학습 및 별도의 테스트 셋으로 예측 성능 평가
              
        clf = RandomForestClassifier(random_state=0 , max_depth = 4 , min_samples_leaf = 2 , min_samples_split =  4 , \
                                     n_estimators = 20 )
        

        clf.fit(X_train , y_train)
        train_pred = clf.predict(X_train_before)
        test_pred = clf.predict(X_test)




        #feature_importance
        feature_importance = clf.feature_importances_


        pred_value = pd.Series(test_pred,index = y_test.index)
        per = test_list[i][y_name]
        y_testdata = pd.concat([per,y_test,pred_value] , axis = 1)
        y_testdata.columns = ['실제 y 수익률','실제 y라벨링','예측 y 라벨링']
        
        train_pred_value = pd.Series(train_pred,index = y_train_before.index)
        per = train_list[i][y_name]
        y_traindata = pd.concat([per,y_train_before,train_pred_value] , axis = 1)
        y_traindata.columns = ['실제 y 수익률','실제 y라벨링','예측 y 라벨링']
        
        
        # 성과 평가
        train_res = get_clf_eval(y_train_before, y_traindata, train_pred)
        test_res = get_clf_eval(y_test, y_testdata,test_pred)
        res = train_res + test_res
        
        score_list.append(res)
        feature_list.append(feature_importance)
        y_data.append(y_testdata)   
    df_score = pd.DataFrame(score_list,columns = ["정확도","정밀도","재현율","f1_score"]+["정확도_test","정밀도_test","재현율_test","f1_score_test"]).T 
    df_score.rename(columns = lambda x : "model_set_"+ str(x),inplace = True)

    df_feature = pd.DataFrame(feature_list,columns = X_train.columns).T 
    df_feature.rename(columns = lambda x : "model_set_"+ str(x),inplace = True)
        
    return df_score , df_feature , y_data

In [232]:
df_score_0_1,df_feature_0_1,y_data_0_1 = process_final(df_0_1,'공모가 대비 1개월 수익률')
df_score_0_1['rlf_mean'] = df_score_0_1.mean(axis=1)
rlf_score = df_score_0_1['rlf_mean']
rlf_score

정확도              0.673916
정밀도              0.706213
재현율              0.792597
f1_score         0.685858
정확도_test         0.329349
정밀도_test         0.479505
재현율_test         0.566559
f1_score_test    0.315526
Name: rlf_mean, dtype: float64

# xgb

In [26]:
def process_final_xgb(df,y_name):
    
    ## 기본 전처리
    df['상장일'] = pd.to_datetime(df['상장일'])
    df = df.set_index(['상장일'])
    df = df.drop(['종목명','공모 시가총액'],axis = 1) ## 나중에 카테고리 진행할려면 남겨줄 것
    if y_name == '공모가 대비 6개월 수익률' or "종가 대비 6개월 수익률":
        df['Cat'] = Cut(df[y_name],[ -0.2, 0.2, 0.4])
       
        
    else :
        df['Cat'] = Cut(df[y_name],[ -0.1, 0.1, 0.2])
       
    
    ## train 3년 test 1년으로 총 기간을 3개월 이동으로 36개 구간분할
    train_list = []
    test_list = []
    train_start_date = '2009-04-01' ## 기한은 나중에 변경할수도
    test_start_date = parse(str(train_start_date)).date() + relativedelta(years =4)
    train_end_date = parse(str(test_start_date)).date() - relativedelta(days =1)
    test_end_date = parse(str(train_end_date)).date() + relativedelta(years =1) 

    while True:

        train_list.append(df[train_start_date : train_end_date])
        test_list.append(df[test_start_date : test_end_date])

        train_start_date = parse(str(train_start_date)).date() + relativedelta(months=3)
        test_start_date = parse(str(test_start_date)).date() + relativedelta(months=3)
        train_end_date = parse(str(train_end_date)).date() + relativedelta(months=3)
        test_end_date = parse(str(test_end_date)).date() + relativedelta(months=3)

        if str(train_start_date) == '2017-04-01':
            break
            
    y_data = []
    score_list = []
    feature_list = []

    for i in range(0,len(train_list)):
        X_train_before = train_list[i].drop([y_name,'Cat'],axis =1)
        y_train_before = train_list[i]['Cat']
        X_test = test_list[i].drop([y_name,'Cat'],axis =1)
        y_test = test_list[i]['Cat']
        
        smote = SMOTE(random_state=0,k_neighbors = 3)
        X_train,y_train = smote.fit_resample(X_train_before,y_train_before)
#         ros = RandomOverSampler()
#         X_train,y_train = ros.fit_resample(X_train_before,y_train_before)

       # 랜덤 포레스트 학습 및 별도의 테스트 셋으로 예측 성능 평가
              
        clf =  XGBClassifier(random_state = 0 ,colsample_bytree = 1, gamma =  0.01,\
                            learning_rate =  0.01, max_depth =  4, reg_lambda =  0, subsample =  1,
                             eval_metric = "mlogloss")
        

        clf.fit(X_train , y_train)
        train_pred = clf.predict(X_train_before)
        test_pred = clf.predict(X_test)




        #feature_importance
        feature_importance = clf.feature_importances_


        pred_value = pd.Series(test_pred,index = y_test.index)
        per = test_list[i][y_name]
        y_testdata = pd.concat([per,y_test,pred_value] , axis = 1)
        y_testdata.columns = ['실제 y 수익률','실제 y라벨링','예측 y 라벨링']
        
        train_pred_value = pd.Series(train_pred,index = y_train_before.index)
        per = train_list[i][y_name]
        y_traindata = pd.concat([per,y_train_before,train_pred_value] , axis = 1)
        y_traindata.columns = ['실제 y 수익률','실제 y라벨링','예측 y 라벨링']
        
        
        # 성과 평가
        train_res = get_clf_eval(y_train_before, y_traindata, train_pred)
        test_res = get_clf_eval(y_test, y_testdata,test_pred)
        res = train_res + test_res
        
        score_list.append(res)
    df_score = pd.DataFrame(score_list,columns = ["정확도","정밀도","재현율","f1_score"]+["정확도_test","정밀도_test","재현율_test","f1_score_test"]).T 
    df_score.rename(columns = lambda x : "model_set_"+ str(x),inplace = True)


        
    return df_score

In [27]:
df_score_0_1_xgb = process_final_xgb(df_0_1,'공모가 대비 1개월 수익률')

In [28]:
df_score_0_1_xgb['xgb_mean'] = df_score_0_1_xgb.mean(axis=1)

In [29]:
xgb_score = df_score_0_1_xgb['xgb_mean']

In [30]:
xgb_score

정확도              0.809976
정밀도              0.813935
재현율              0.899144
f1_score         0.815789
정확도_test         0.318096
정밀도_test         0.457703
재현율_test         0.498978
f1_score_test    0.308028
Name: xgb_mean, dtype: float64

# lgbm

In [176]:
def process_final_lgbm(df,y_name):
    
    ## 기본 전처리
    df['상장일'] = pd.to_datetime(df['상장일'])
    df = df.set_index(['상장일'])
    df = df.drop(['종목명','공모 시가총액'],axis = 1) ## 나중에 카테고리 진행할려면 남겨줄 것
    if y_name == '공모가 대비 6개월 수익률' or "종가 대비 6개월 수익률":
        df['Cat'] = Cut(df[y_name],[ -0.2, 0.2, 0.4])
       
        
    else :
        df['Cat'] = Cut(df[y_name],[ -0.1, 0.1, 0.2])
       
    
    ## train 3년 test 1년으로 총 기간을 3개월 이동으로 36개 구간분할
    train_list = []
    test_list = []
    train_start_date = '2009-04-01' ## 기한은 나중에 변경할수도
    test_start_date = parse(str(train_start_date)).date() + relativedelta(years =4)
    train_end_date = parse(str(test_start_date)).date() - relativedelta(days =1)
    test_end_date = parse(str(train_end_date)).date() + relativedelta(years =1) 

    while True:

        train_list.append(df[train_start_date : train_end_date])
        test_list.append(df[test_start_date : test_end_date])

        train_start_date = parse(str(train_start_date)).date() + relativedelta(months=3)
        test_start_date = parse(str(test_start_date)).date() + relativedelta(months=3)
        train_end_date = parse(str(train_end_date)).date() + relativedelta(months=3)
        test_end_date = parse(str(test_end_date)).date() + relativedelta(months=3)

        if str(train_start_date) == '2017-04-01':
            break
            
    y_data = []
    score_list = []
    feature_list = []

    for i in range(0,len(train_list)):
        X_train_before = train_list[i].drop([y_name,'Cat'],axis =1)
        y_train_before = train_list[i]['Cat']
        X_test = test_list[i].drop([y_name,'Cat'],axis =1)
        y_test = test_list[i]['Cat']
        
        smote = SMOTE(random_state=0,k_neighbors = 3)
        X_train,y_train = smote.fit_resample(X_train_before,y_train_before)
#         ros = RandomOverSampler()
#         X_train,y_train = ros.fit_resample(X_train_before,y_train_before)

       # 랜덤 포레스트 학습 및 별도의 테스트 셋으로 예측 성능 평가
              
        clf =  LGBMClassifier(random_state =0, min_data_in_leaf = 20 , reg_alpha = 0.1,n_estimators=30,max_depth = 5)


        clf.fit(X_train , y_train)
        train_pred = clf.predict(X_train_before)
        test_pred = clf.predict(X_test)




        #feature_importance
        feature_importance = clf.feature_importances_


        pred_value = pd.Series(test_pred,index = y_test.index)
        per = test_list[i][y_name]
        y_testdata = pd.concat([per,y_test,pred_value] , axis = 1)
        y_testdata.columns = ['실제 y 수익률','실제 y라벨링','예측 y 라벨링']
        
        train_pred_value = pd.Series(train_pred,index = y_train_before.index)
        per = train_list[i][y_name]
        y_traindata = pd.concat([per,y_train_before,train_pred_value] , axis = 1)
        y_traindata.columns = ['실제 y 수익률','실제 y라벨링','예측 y 라벨링']
        
        
        # 성과 평가
        train_res = get_clf_eval(y_train_before, y_traindata, train_pred)
        test_res = get_clf_eval(y_test, y_testdata,test_pred)
        res = train_res + test_res
        
        score_list.append(res)
    df_score = pd.DataFrame(score_list,columns = ["정확도","정밀도","재현율","f1_score"]+["정확도_test","정밀도_test","재현율_test","f1_score_test"]).T 
    df_score.rename(columns = lambda x : "model_set_"+ str(x),inplace = True)


        
    return df_score

In [177]:
df_score_0_1_lgbm = process_final_lgbm(df_0_1,'공모가 대비 1개월 수익률')

[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=20
[LightGBM] [Warning] min

In [178]:
df_score_0_1_lgbm.mean(axis=1)
df_score_0_1_lgbm['lgbm_mean'] = df_score_0_1_lgbm.mean(axis=1)
lgbm_score = df_score_0_1_lgbm['lgbm_mean']

# 결과값

In [179]:
df = pd.concat([rlf_score,xgb_score,lgbm_score],axis=1)

In [180]:
df

,rlf_mean,xgb_mean,lgbm_mean
정확도,0.632206,0.809976,0.936390
정밀도,0.678056,0.813935,0.942048
재현율,0.762882,0.899144,0.945862
f1_score,0.643590,0.815789,0.936847
정확도_test,0.325699,0.318096,0.325616
정밀도_test,0.480501,0.457703,0.421926
재현율_test,0.573647,0.498978,0.476593
f1_score_test,0.308471,0.308028,0.299252


In [181]:
df.to_csv("모델별 평가지표 비교.csv",encoding = 'euc-kr')

## SMOTE VS ROS

In [37]:
def process_final_ros(df,y_name):
    
    ## 기본 전처리
    df['상장일'] = pd.to_datetime(df['상장일'])
    df = df.set_index(['상장일'])
    df = df.drop(['종목명','공모 시가총액'],axis = 1) ## 나중에 카테고리 진행할려면 남겨줄 것
    if y_name == '공모가 대비 6개월 수익률' or "종가 대비 6개월 수익률":
        df['Cat'] = Cut(df[y_name],[ -0.2, 0.2, 0.4])
       
        
    else :
        df['Cat'] = Cut(df[y_name],[ -0.1, 0.1, 0.2])
       
    
    ## train 3년 test 1년으로 총 기간을 3개월 이동으로 36개 구간분할
    train_list = []
    test_list = []
    train_start_date = '2009-04-01' ## 기한은 나중에 변경할수도
    test_start_date = parse(str(train_start_date)).date() + relativedelta(years =4)
    train_end_date = parse(str(test_start_date)).date() - relativedelta(days =1)
    test_end_date = parse(str(train_end_date)).date() + relativedelta(years =1) 

    while True:

        train_list.append(df[train_start_date : train_end_date])
        test_list.append(df[test_start_date : test_end_date])

        train_start_date = parse(str(train_start_date)).date() + relativedelta(months=3)
        test_start_date = parse(str(test_start_date)).date() + relativedelta(months=3)
        train_end_date = parse(str(train_end_date)).date() + relativedelta(months=3)
        test_end_date = parse(str(test_end_date)).date() + relativedelta(months=3)

        if str(train_start_date) == '2017-04-01':
            break
            
    y_data = []
    score_list = []
    feature_list = []

    for i in range(0,len(train_list)):
        X_train_before = train_list[i].drop([y_name,'Cat'],axis =1)
        y_train_before = train_list[i]['Cat']
        X_test = test_list[i].drop([y_name,'Cat'],axis =1)
        y_test = test_list[i]['Cat']
        
#         smote = SMOTE(random_state=0,k_neighbors = 3)
#         X_train,y_train = smote.fit_resample(X_train_before,y_train_before)
        ros = RandomOverSampler()
        X_train,y_train = ros.fit_resample(X_train_before,y_train_before)

       # 랜덤 포레스트 학습 및 별도의 테스트 셋으로 예측 성능 평가
              
        clf = RandomForestClassifier(random_state=0 , max_depth = 5 , min_samples_leaf = 6 , min_samples_split =  12 , \
                                     n_estimators = 20 )
        

        clf.fit(X_train , y_train)
        train_pred = clf.predict(X_train_before)
        test_pred = clf.predict(X_test)




        #feature_importance
        feature_importance = clf.feature_importances_


        pred_value = pd.Series(test_pred,index = y_test.index)
        per = test_list[i][y_name]
        y_testdata = pd.concat([per,y_test,pred_value] , axis = 1)
        y_testdata.columns = ['실제 y 수익률','실제 y라벨링','예측 y 라벨링']
        
        train_pred_value = pd.Series(train_pred,index = y_train_before.index)
        per = train_list[i][y_name]
        y_traindata = pd.concat([per,y_train_before,train_pred_value] , axis = 1)
        y_traindata.columns = ['실제 y 수익률','실제 y라벨링','예측 y 라벨링']
        
        
        # 성과 평가
        train_res = get_clf_eval(y_train_before, y_traindata, train_pred)
        test_res = get_clf_eval(y_test, y_testdata,test_pred)
        res = train_res + test_res
        
        score_list.append(res)
        feature_list.append(feature_importance)
        y_data.append(y_testdata)   
    df_score = pd.DataFrame(score_list,columns = ["정확도","정밀도","재현율","f1_score"]+["정확도_test","정밀도_test","재현율_test","f1_score_test"]).T 
    df_score.rename(columns = lambda x : "model_set_"+ str(x),inplace = True)


        
    return df_score 

In [38]:
df_score_0_1_ros = process_final_ros(df_0_1,'공모가 대비 1개월 수익률')

In [54]:
df_score_0_1_ros['ros_mean'] = df_score_0_1_ros.mean(axis=1)

In [55]:
ros_score = df_score_0_1_ros['ros_mean']

In [57]:
data = pd.concat([rlf_score,ros_score],axis = 1)

In [60]:
data = data.rename(columns = {'rlf_mean':"smote_mean"})

In [61]:
data

,smote_mean,ros_mean
정확도,0.702678,0.726614
정밀도,0.726528,0.736517
재현율,0.805926,0.835657
f1_score,0.709760,0.730981
정확도_test,0.322484,0.324001
정밀도_test,0.476976,0.469909
재현율_test,0.558717,0.612459
f1_score_test,0.304779,0.308602


In [63]:
data.to_csv("smote_ros 비교.csv",encoding = 'euc-kr')

## 클래스를 3개로 했을때

In [84]:
def process_final_3(df,y_name):
    
    ## 기본 전처리
    df['상장일'] = pd.to_datetime(df['상장일'])
    df = df.set_index(['상장일'])
    df = df.drop(['종목명','공모 시가총액'],axis = 1) ## 나중에 카테고리 진행할려면 남겨줄 것
    if y_name == '공모가 대비 6개월 수익률' or "종가 대비 6개월 수익률":
        df['Cat'] = Cut(df[y_name],[ 0])
       
        
    else :
        df['Cat'] = Cut(df[y_name],[ 0])
       
    
    ## train 3년 test 1년으로 총 기간을 3개월 이동으로 36개 구간분할
    train_list = []
    test_list = []
    train_start_date = '2009-04-01' ## 기한은 나중에 변경할수도
    test_start_date = parse(str(train_start_date)).date() + relativedelta(years =4)
    train_end_date = parse(str(test_start_date)).date() - relativedelta(days =1)
    test_end_date = parse(str(train_end_date)).date() + relativedelta(years =1) 

    while True:

        train_list.append(df[train_start_date : train_end_date])
        test_list.append(df[test_start_date : test_end_date])

        train_start_date = parse(str(train_start_date)).date() + relativedelta(months=3)
        test_start_date = parse(str(test_start_date)).date() + relativedelta(months=3)
        train_end_date = parse(str(train_end_date)).date() + relativedelta(months=3)
        test_end_date = parse(str(test_end_date)).date() + relativedelta(months=3)

        if str(train_start_date) == '2017-04-01':
            break
            
    y_data = []
    score_list = []
    feature_list = []

    for i in range(0,len(train_list)):
        X_train_before = train_list[i].drop([y_name,'Cat'],axis =1)
        y_train_before = train_list[i]['Cat']
        X_test = test_list[i].drop([y_name,'Cat'],axis =1)
        y_test = test_list[i]['Cat']
        
        smote = SMOTE(random_state=0)
        X_train,y_train = smote.fit_resample(X_train_before,y_train_before)
#         ros = RandomOverSampler()
#         X_train,y_train = ros.fit_resample(X_train_before,y_train_before)

       # 랜덤 포레스트 학습 및 별도의 테스트 셋으로 예측 성능 평가
              
        clf = RandomForestClassifier(random_state=0 , max_depth = 5 , min_samples_leaf = 6 , min_samples_split =  12 , \
                                     n_estimators = 20 )
        

        clf.fit(X_train , y_train)
        train_pred = clf.predict(X_train_before)
        test_pred = clf.predict(X_test)




        #feature_importance
        feature_importance = clf.feature_importances_


        pred_value = pd.Series(test_pred,index = y_test.index)
        per = test_list[i][y_name]
        y_testdata = pd.concat([per,y_test,pred_value] , axis = 1)
        y_testdata.columns = ['실제 y 수익률','실제 y라벨링','예측 y 라벨링']
        
        train_pred_value = pd.Series(train_pred,index = y_train_before.index)
        per = train_list[i][y_name]
        y_traindata = pd.concat([per,y_train_before,train_pred_value] , axis = 1)
        y_traindata.columns = ['실제 y 수익률','실제 y라벨링','예측 y 라벨링']
        
        
        # 성과 평가
        train_res = get_clf_eval(y_train_before, y_traindata, train_pred)
        test_res = get_clf_eval(y_test, y_testdata,test_pred)
        res = train_res + test_res
        
        score_list.append(res)
        feature_list.append(feature_importance)
        y_data.append(y_testdata)   
    df_score = pd.DataFrame(score_list,columns = ["정확도","정밀도","재현율","f1_score"]+["정확도_test","정밀도_test","재현율_test","f1_score_test"]).T 
    df_score.rename(columns = lambda x : "model_set_"+ str(x),inplace = True)

    df_feature = pd.DataFrame(feature_list,columns = X_train.columns).T 
    df_feature.rename(columns = lambda x : "model_set_"+ str(x),inplace = True)
        
    return df_score , df_feature , y_data

In [85]:
df_score_3,df_feature_3,y_data_3 = process_final_3(df_0_1,'공모가 대비 1개월 수익률')

In [86]:
df_score_3['by3_mean'] = df_score_3.mean(axis=1)

In [87]:
df3_score = df_score_3['by3_mean']

In [89]:
class_34 = pd.concat([rlf_score,df3_score],axis=1)

In [90]:
class_34.to_csv("class 비교.csv",encoding = 'euc-kr')